In [2]:
import tensorflow as tf
from time import time
import numpy
import os
import json
import pickle
import pandas
import datetime
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import data_pipeline
import conv_model
import initialize
import prepare_data
import flacdb
import plot_batch
import loss_metrics
import generate_report

In [3]:
! nvidia-smi

Tue Nov 19 10:14:45 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.78       Driver Version: 410.78       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX TIT...  On   | 00000000:89:00.0 Off |                  N/A |
| 22%   33C    P8    16W / 250W |    116MiB / 12212MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
%%time

H = initialize.load_hypes()
initial_data_path = '/scr1/mimic/initial_data/'

# sig_data, metadata = initialize.load_initial_data(save_path=initial_data_path)
sig_data, metadata = initialize.load_initial_data(load_path=initial_data_path)
diagnosis = initialize.load_diagnosis(H['icd_codes'], metadata)
diagnosis = initialize.augment_diagnosis(diagnosis, metadata)
diagnosis = initialize.fix_diagnosis(diagnosis)
priors = (diagnosis == 1).sum() / (diagnosis != 0).sum()
priors['measured_systemic_hypertension'] = 0.5
priors['measured_pulmonary_hypertension'] = 0.5
diagnosis = initialize.conform_diagnosis(diagnosis, metadata)
partition = initialize.load_partition(H, sig_data)

CPU times: user 3.17 s, sys: 316 ms, total: 3.49 s
Wall time: 3.55 s


In [42]:
importlib.reload(data_pipeline)

<module 'data_pipeline' from '../libs/data_pipeline.py'>

In [43]:
%%time

I = partition['validation']
row_lengths = initialize.get_row_lengths(metadata[I])
args = [metadata[I], sig_data[I], diagnosis[I], row_lengths]
tensors = initialize.get_tensors(H, *args)
dataset = data_pipeline.build(H, tensors, 'train')

CPU times: user 2.19 s, sys: 28 ms, total: 2.22 s
Wall time: 2.2 s


In [44]:
x, y = next(iter(dataset))

In [48]:
y['pressure'].numpy().min()

-2.9990134

In [9]:
importlib.reload(conv_model)

<module 'conv_model' from '../libs/conv_model.py'>

In [10]:
%%time

model = conv_model.build(H, priors)

CPU times: user 23.6 s, sys: 592 ms, total: 24.2 s
Wall time: 24 s


In [11]:
! ls /scr1/checkpoints/

1117408_20191117-112416.data-00000-of-00002
1117408_20191117-112416.data-00001-of-00002
1117408_20191117-112416.index
1118450_20191117-135257.data-00000-of-00002
1118450_20191117-135257.data-00001-of-00002
1118450_20191117-135257.index
1124961_20191118-10101100100.ckpt.data-00000-of-00002
1124961_20191118-10101100100.ckpt.data-00001-of-00002
1124961_20191118-10101100100.ckpt.index
1124961_20191118-10101100200.ckpt.data-00000-of-00002
1124961_20191118-10101100200.ckpt.data-00001-of-00002
1124961_20191118-10101100200.ckpt.index
1124961_20191118-10101100300.ckpt.data-00000-of-00002
1124961_20191118-10101100300.ckpt.data-00001-of-00002
1124961_20191118-10101100300.ckpt.index
1124961_20191118-10101100400.ckpt.data-00000-of-00002
1124961_20191118-10101100400.ckpt.data-00001-of-00002
1124961_20191118-10101100400.ckpt.index
1124961_20191118-10101100500.ckpt.data-00000-of-00002
1124961_20191118-10101100500.ckpt.data-00001-of-00002
1124961_20191118-10101100500.ckpt.index
1124961_20191118-1010110

In [12]:
checkpoint_path = '/scr1/checkpoints/1127960_20191119-055644_00200.ckpt'
model.load_weights(checkpoint_path)

In [14]:
%%time

batch_count = 2**(16 - H['batch_size_log2'])
fix_input = lambda x: {**x, 'mask': tf.cast(x['mask'], 'float')}
X, Y, P = generate_report.compute_predictions(model, dataset, batch_count, fix_input)

CPU times: user 24min 15s, sys: 31.2 s, total: 24min 46s
Wall time: 14min 36s


In [36]:
importlib.reload(generate_report)

<module 'generate_report' from '../libs/generate_report.py'>

In [24]:
numpy.expand_dims

<function numpy.expand_dims(a, axis)>

In [27]:
H['input_sigs']

['PLETH', 'RESP', 'I', 'II', 'III', 'V', 'AVR', 'MCL', 'MCL1', 'AVL', 'AVF']

In [25]:
(X['signals'] * numpy.expand_dims(X['mask'], 1)).shape

(63618, 512, 11)

In [28]:
numpy.abs(X['signals'] * numpy.expand_dims(X['mask'], 1)).sum(0).sum(0)

array([3.6948920e+07, 6.4749845e+06, 3.0315431e+05, 1.0935051e+07,
       1.3603430e+06, 1.4337848e+07, 1.2710098e+07, 1.3048426e+06,
       0.0000000e+00, 1.4027549e+03, 5.4376707e+04], dtype=float32)

In [15]:
%matplotlib widget
codes = diagnosis.columns.to_list()
codes += ['measured_systemic_hypertension', 'measured_pulmonary_hypertension']
y_true, y_pred = Y['diagnosis'], P['diagnosis']
plot_diagnoses = generate_report.get_diagnoses_plotter(y_true, y_pred, codes)
plot_diagnoses();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='i', max=42), Output()), _dom_classes=('widget-interact',…

In [313]:
importlib.reload(generate_report)

<module 'generate_report' from '../libs/generate_report.py'>